In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [4]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)


standardized_market_caps = standardized_market_caps.loc[standardized_mom.index]
standardized_BtoM = standardized_BtoM.loc[standardized_mom.index]
standardized_mom = standardized_mom.loc[standardized_mom.index]
market_weights = market_weights.loc[standardized_mom.index]
df_48ind = df_48ind.loc[standardized_mom.index]

df_ret = df_48ind.shift(-1).dropna()

standardized_market_caps = standardized_market_caps.loc[df_ret.index]
standardized_BtoM = standardized_BtoM.loc[df_ret.index]
standardized_mom = standardized_mom.loc[df_ret.index]
market_weights = market_weights.loc[df_ret.index]
df_48ind_ret = df_48ind.loc[df_ret.index]

def calculate_regression_coefficients(N, returns, characteristic1, characteristic2, characteristic3):

    model = LinearRegression()

    beta_initial = []

    for i in range(N):
        y = returns.iloc[:, i]
        X1 = characteristic1.iloc[:, i]
        X2 = characteristic2.iloc[:, i]
        X3 = characteristic3.iloc[:, i]

        X_combined = pd.concat([X1, X2, X3], axis=1)

        model.fit(X_combined, y)
        beta_initial.append(model.coef_)

    return np.mean(beta_initial, axis=0)

coefficients_moyens = calculate_regression_coefficients(df_48ind.shape[1], df_48ind_ret, standardized_market_caps, standardized_BtoM, standardized_mom)


def Expanding_window_optimization(MC, BM, MOM, r, gamma=5):
    
    def u(x, gamma=5):
        return (1+x) ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma=5):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum((wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r),axis=1), gamma))

    optimal_betas = {}  # Define optimal_betas here to store the optimal coefficients for each period
    
    start_year = df_48ind.index.min() # Date de début fixée à la première date dans les données de rendement
    
    # Date de fin initiale fixée à décembre 1973
    initial_end_year = pd.to_datetime('1973-12-01')
    # Date de fin courante, commence par initial_end_year et sera ajustée dans la boucle
    current_end_year = initial_end_year

    # Initial guess for theta
    initial_beta = coefficients_moyens

    while current_end_year <= df_48ind.index.max(): # Boucle jusqu'à la dernière date dans les données de rendement 
        
        # Extraire les composantes nécessaires pour l'optimisation des coefficients pour la période sélectionnée dans la window
        MC = standardized_market_caps.loc[start_year:current_end_year]
        BM = standardized_BtoM.loc[start_year:current_end_year]
        MOM = standardized_mom.loc[start_year:current_end_year]
        r = df_48ind.loc[start_year:current_end_year]
            
        N = df_48ind.shape[1]
        T = df_48ind.shape[0]

        # Run the optimization for the current period
        result = minimize(function_to_maximize, initial_beta, args=( MC, BM, MOM, r, T, N, gamma), method='SLSQP')

        # Store the optimal theta for the current period
        if result.success:
            optimal_betas[current_end_year] = result.x
            initial_beta = result.x  # Update initial_beta with the optimal betas
        else:
            optimal_betas[current_end_year] = None
    

        # Élargir la fenêtre pour la prochaine itération de window en ajoutant 12 mois à la dernière date courante de fin 
        current_end_year += pd.DateOffset(months=12)

    return optimal_betas

In [5]:
Expanding_window_optimization(standardized_market_caps, standardized_BtoM, standardized_mom, df_ret)

{Timestamp('1973-12-01 00:00:00'): array([0.54327476, 0.57571289, 2.19551398]),
 Timestamp('1974-12-01 00:00:00'): array([0.39056251, 0.23222941, 2.2230283 ]),
 Timestamp('1975-12-01 00:00:00'): array([0.39711377, 0.22277083, 2.2248954 ]),
 Timestamp('1976-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1977-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1978-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1979-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1980-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1981-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1982-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1983-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1984-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -